In [38]:
import numpy as np
import string
import pandas as pd
import nltk
import keras
import os

from sklearn.random_projection import GaussianRandomProjection
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold, train_test_split
from sklearn.utils import shuffle
from sklearn import preprocessing, svm#scale, regresions, cross shuffle stats sepeareate data

from nltk.corpus import stopwords

from keras.models import Model
from keras.layers import *
from keras.optimizers import Adam
from keras import metrics
from keras.callbacks import ModelCheckpoint, EarlyStopping

import tensorflow as tf

import matplotlib as mpl
import matplotlib.pyplot as plt
import os


# modify the data folder accordingly
data_folder = "data/"
cache_folder = "cache/"

stop_words = set(stopwords.words("english") + list(string.punctuation))

## Import Price Data

In [39]:
price = pd.read_csv("data/ETH_price.csv")

In [40]:
price

,Date,Symbol,Open,High,Low,Close,Volume ETH,Volume USD
0,2020-04-01 11-AM,ETHUSD,132.17,132.17,132.02,132.02,0.00,0.00
1,2020-04-01 10-AM,ETHUSD,132.20,132.29,131.94,132.17,2509.86,331220.30
2,2020-04-01 09-AM,ETHUSD,132.28,132.32,131.54,132.20,203.96,26906.66
3,2020-04-01 08-AM,ETHUSD,132.55,132.58,131.68,132.28,802.42,106059.43
4,2020-04-01 07-AM,ETHUSD,132.46,134.49,132.08,132.55,2955.84,393453.46
5,2020-04-01 06-AM,ETHUSD,132.15,132.82,131.81,132.46,2849.55,376991.17
6,2020-04-01 05-AM,ETHUSD,133.24,133.24,131.93,132.15,2927.83,387028.90
7,2020-04-01 04-AM,ETHUSD,133.27,133.35,132.89,133.24,65.65,8740.67
8,2020-04-01 03-AM,ETHUSD,132.48,133.39,132.39,133.27,493.35,65700.68
9,2020-04-01 02-AM,ETHUSD,131.77,132.48,131.49,132.48,60.21,7949.72


In [41]:
new_date = []
for t in price["Date"]:
    t0 = ""
    if t[-2:] == "AM":
        if t[-5:-3] == "12":
            t0 = t[:-5] + "00:00"
        else:
            t0 = t[:-5] + t[-5:-3] + ":00"           
    else:
        if t[-5:-3] == "12":
            t0 = t[:-5] + "12:00"
        else:
            t0 = t[:-5] + str(int(t[-5:-3])+12)+ ":00"
        
    new_date.append(t0)
        
price["Date"] = new_date

In [42]:
price["Date"][0][-5:-3]

'11'

In [43]:
price

,Date,Symbol,Open,High,Low,Close,Volume ETH,Volume USD
0,2020-04-01 11:00,ETHUSD,132.17,132.17,132.02,132.02,0.00,0.00
1,2020-04-01 10:00,ETHUSD,132.20,132.29,131.94,132.17,2509.86,331220.30
2,2020-04-01 09:00,ETHUSD,132.28,132.32,131.54,132.20,203.96,26906.66
3,2020-04-01 08:00,ETHUSD,132.55,132.58,131.68,132.28,802.42,106059.43
4,2020-04-01 07:00,ETHUSD,132.46,134.49,132.08,132.55,2955.84,393453.46
5,2020-04-01 06:00,ETHUSD,132.15,132.82,131.81,132.46,2849.55,376991.17
6,2020-04-01 05:00,ETHUSD,133.24,133.24,131.93,132.15,2927.83,387028.90
7,2020-04-01 04:00,ETHUSD,133.27,133.35,132.89,133.24,65.65,8740.67
8,2020-04-01 03:00,ETHUSD,132.48,133.39,132.39,133.27,493.35,65700.68
9,2020-04-01 02:00,ETHUSD,131.77,132.48,131.49,132.48,60.21,7949.72


In [44]:
price["Date"] = pd.to_datetime(price.Date)

In [45]:
price

,Date,Symbol,Open,High,Low,Close,Volume ETH,Volume USD
0,2020-04-01 11:00:00,ETHUSD,132.17,132.17,132.02,132.02,0.00,0.00
1,2020-04-01 10:00:00,ETHUSD,132.20,132.29,131.94,132.17,2509.86,331220.30
2,2020-04-01 09:00:00,ETHUSD,132.28,132.32,131.54,132.20,203.96,26906.66
3,2020-04-01 08:00:00,ETHUSD,132.55,132.58,131.68,132.28,802.42,106059.43
4,2020-04-01 07:00:00,ETHUSD,132.46,134.49,132.08,132.55,2955.84,393453.46
5,2020-04-01 06:00:00,ETHUSD,132.15,132.82,131.81,132.46,2849.55,376991.17
6,2020-04-01 05:00:00,ETHUSD,133.24,133.24,131.93,132.15,2927.83,387028.90
7,2020-04-01 04:00:00,ETHUSD,133.27,133.35,132.89,133.24,65.65,8740.67
8,2020-04-01 03:00:00,ETHUSD,132.48,133.39,132.39,133.27,493.35,65700.68
9,2020-04-01 02:00:00,ETHUSD,131.77,132.48,131.49,132.48,60.21,7949.72


In [46]:
price.sort_values(by=['Date'],inplace = True, ascending=True)

In [47]:
price

,Date,Symbol,Open,High,Low,Close,Volume ETH,Volume USD
20911,2017-09-22 19:00:00,ETHUSD,729.73,755.00,604.19,639.99,52863.64,35707764.33
20910,2017-11-12 05:00:00,ETHUSD,312.34,316.34,308.00,310.16,3263.16,1017470.49
20909,2017-11-12 06:00:00,ETHUSD,310.16,310.16,301.12,307.00,3463.15,1056163.67
20908,2017-11-12 07:00:00,ETHUSD,307.00,314.37,301.37,307.16,6681.07,2066540.88
20907,2017-11-12 08:00:00,ETHUSD,307.16,310.00,302.48,304.79,2152.84,656841.26
20906,2017-11-12 09:00:00,ETHUSD,304.79,305.00,300.38,302.40,2832.44,857577.88
20905,2017-11-12 10:00:00,ETHUSD,302.40,308.09,300.11,305.18,2169.39,657893.16
20904,2017-11-12 11:00:00,ETHUSD,305.18,306.79,301.36,303.88,1115.28,339483.77
20903,2017-11-12 12:00:00,ETHUSD,303.88,305.43,301.37,301.59,1903.05,577686.54
20902,2017-11-12 13:00:00,ETHUSD,301.59,306.98,301.22,304.83,2469.31,749944.26


In [48]:
price.drop([20911],inplace=True)

In [49]:
price.reset_index(drop=True,inplace=True)

In [50]:
price

,Date,Symbol,Open,High,Low,Close,Volume ETH,Volume USD
0,2017-11-12 05:00:00,ETHUSD,312.34,316.34,308.00,310.16,3263.16,1017470.49
1,2017-11-12 06:00:00,ETHUSD,310.16,310.16,301.12,307.00,3463.15,1056163.67
2,2017-11-12 07:00:00,ETHUSD,307.00,314.37,301.37,307.16,6681.07,2066540.88
3,2017-11-12 08:00:00,ETHUSD,307.16,310.00,302.48,304.79,2152.84,656841.26
4,2017-11-12 09:00:00,ETHUSD,304.79,305.00,300.38,302.40,2832.44,857577.88
5,2017-11-12 10:00:00,ETHUSD,302.40,308.09,300.11,305.18,2169.39,657893.16
6,2017-11-12 11:00:00,ETHUSD,305.18,306.79,301.36,303.88,1115.28,339483.77
7,2017-11-12 12:00:00,ETHUSD,303.88,305.43,301.37,301.59,1903.05,577686.54
8,2017-11-12 13:00:00,ETHUSD,301.59,306.98,301.22,304.83,2469.31,749944.26
9,2017-11-12 14:00:00,ETHUSD,304.83,305.80,301.00,305.38,1936.74,587803.84


In [51]:
perc = []
change = []
i=0

for i in range(len(price["Close"])):
    if i == 0:
        p = 0
    elif price["Close"][i-1]==0:
        p = 0
    else:
        p = (price["Close"][i] - price["Close"][i-1])/price["Close"][i-1]
    
    perc.append(p)
    
    if i == 0:
        c = 0
    elif price["Close"][i-1]==price["Close"][i]:
        c = 0
    elif price["Close"][i-1]<price["Close"][i]:
        c = 1
    else:
        c = -1
    
    change.append(c)
    
price["percentage"] = perc
price["change"] = change
    

In [52]:
price

,Date,Symbol,Open,High,Low,Close,Volume ETH,Volume USD,percentage,change
0,2017-11-12 05:00:00,ETHUSD,312.34,316.34,308.00,310.16,3263.16,1017470.49,0.000000,0
1,2017-11-12 06:00:00,ETHUSD,310.16,310.16,301.12,307.00,3463.15,1056163.67,-0.010188,-1
2,2017-11-12 07:00:00,ETHUSD,307.00,314.37,301.37,307.16,6681.07,2066540.88,0.000521,1
3,2017-11-12 08:00:00,ETHUSD,307.16,310.00,302.48,304.79,2152.84,656841.26,-0.007716,-1
4,2017-11-12 09:00:00,ETHUSD,304.79,305.00,300.38,302.40,2832.44,857577.88,-0.007841,-1
5,2017-11-12 10:00:00,ETHUSD,302.40,308.09,300.11,305.18,2169.39,657893.16,0.009193,1
6,2017-11-12 11:00:00,ETHUSD,305.18,306.79,301.36,303.88,1115.28,339483.77,-0.004260,-1
7,2017-11-12 12:00:00,ETHUSD,303.88,305.43,301.37,301.59,1903.05,577686.54,-0.007536,-1
8,2017-11-12 13:00:00,ETHUSD,301.59,306.98,301.22,304.83,2469.31,749944.26,0.010743,1
9,2017-11-12 14:00:00,ETHUSD,304.83,305.80,301.00,305.38,1936.74,587803.84,0.001804,1


In [53]:
type(price)

pandas.core.frame.DataFrame

## Import Sentiment Data

In [54]:
comms = pd.read_csv("data/Comms_data.csv", encoding="latin1")

In [55]:
comms = comms.loc[:, ~comms.columns.str.contains('^Unnamed')]

In [56]:
comms

,score,id,body,timestamp
0,0,dpoqn18,Things like trillion dollar bail outs for bank...,2017-11-12 08:28:00
1,63,dpox4a4,Let's all chip in and buy it as a gift? Who is...,2017-11-12 10:47:00
2,2,dpoz0dc,Nice! Thanks for the morning chuckle!,2017-11-12 11:26:00
3,1,dpozit2,I find it interesting all the fud being levele...,2017-11-12 11:36:00
4,3,dpp35gn,"Cheers, Butterfly. The woods are lovely, dark,...",2017-11-12 13:02:00
5,9,dpp57uq,I'd donate. It's about time we gave something ...,2017-11-12 14:00:00
6,2,dpp5fsl,Haha. Great post. I think Vitalik would apprec...,2017-11-12 14:07:00
7,2,dpp65dj,but where is Ceasar the Cat with glasses???\n,2017-11-12 14:30:00
8,1,dppczdp,He doesn¨¹t have a car?!,2017-11-12 19:41:00
9,0,dppe59z,Is he supposed to sit on the hood or ?,2017-11-12 20:35:00


In [57]:
#Data Analysis
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#Data Preprocessing and Feature Engineering
from textblob import TextBlob
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

In [58]:
def form_sentence(tweet):
    tweet_blob = TextBlob(tweet)
    return ' '.join(tweet_blob.words)

In [59]:
def no_user_alpha(tweet):
    tweet_list = [ele for ele in tweet.split() if ele != 'user']
    clean_tokens = [t for t in tweet_list if re.match(r'[^\W\d]*$', t)]
    clean_s = ' '.join(clean_tokens)
    clean_mess = [word for word in clean_s.split() if word.lower() not in stopwords.words('english')]
    return clean_mess

In [60]:
def normalization(tweet_list):
        lem = WordNetLemmatizer()
        normalized_tweet = []
        for word in tweet_list:
            normalized_text = lem.lemmatize(word,'v')
            normalized_tweet.append(normalized_text)
        return normalized_tweet

In [ ]:
def clean_list(list_t):
    clean_train = []
    
    for t in list_t:
        t = str(t)
        clean_t = form_sentence(t)
        clean_t = no_user_alpha(clean_t)
        clean_t = normalization(clean_t)
        listToStr = ' '.join([str(elem) for elem in clean_t])
        clean_train.append(listToStr)
        
    return clean_train

In [ ]:
clean_title = clean_list(comms["body"])

Exception ignored in: <function SeekableUnicodeStreamReader.__del__ at 0x0000012C43344510>
Traceback (most recent call last):
  File "C:\Users\litia\AppData\Local\Continuum\anaconda3\lib\site-packages\nltk\data.py", line 1281, in __del__
    self.close()
  File "C:\Users\litia\AppData\Local\Continuum\anaconda3\lib\site-packages\nltk\data.py", line 1310, in close
    self.stream.close()
KeyboardInterrupt


In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

sia = SIA()
results = []

headlines = clean_title

for line in headlines:
    pol_score = sia.polarity_scores(line)
    pol_score['headline'] = line
    results.append(pol_score['compound'])

print(results[:3])

In [ ]:
comms['Sentiment Index'] = results

In [ ]:
comms

## Matching Price with Sentiment

In [ ]:
price

In [ ]:
comms.to_csv("data/Raw_data.csv")

In [ ]:
Sentiment = []
total = []
counter = 0

for line in price['Date']:
    x = 0
    
    sent = 0
    pos_m = 0
    neg_m = 0
    for i in range(0,len(comms)):
        t = str(comms['timestamp'][i])
        if(str(line)[0:4] == t[0:4] and str(line)[5:7] == t[5:7] and str(line)[8:10] == t[8:10] and str(line)[11:13] == t[11:13]):
            m = comms['Sentiment Index'][i]
            if m>0:
                pos_m = pos_m + m
            else:
                neg_m = neg_m - m
            
            x = x+1
        if(x!=0 and t[11:13] != str(line)[11:13]):
            break

    if(x!=0):
        sent = (pos_m - neg_m)/(pos_m + neg_m)
        Sentiment.append(sent)
    else:
        Sentiment.append(0)
    
    s_2 = (pos_m+1)/(1+neg_m)
    total.append(np.log(s_2))
    counter = counter+1
    print(counter)

In [ ]:
comms['timestamp'][i]

In [ ]:
#del Sentiment[-1]

In [ ]:
price['Sentiment_1'] = Sentiment
price['Sentiment_2'] = total

In [ ]:
price

In [ ]:
price.to_csv("data/Raw_data.csv")